<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/HVAC_%EC%B5%9C%EC%A0%81%ED%99%94_%EC%8B%9C%EC%8A%A4%ED%85%9C_Python_%EC%A0%84%EC%B2%B4_%EC%BD%94%EB%93%9C_(%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5_%ED%8F%AC%ED%95%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# import joblib # 실제 모델 저장/로드를 위해 사용 가능

# --- 더미 데이터 생성 함수 ---
def generate_dummy_data(num_rows=1000):
    """학습 및 테스트를 위한 더미 시계열 데이터를 생성합니다."""
    data = pd.DataFrame()
    data['datetime'] = pd.to_datetime('2023-01-01') + pd.to_timedelta(np.arange(num_rows) * 1, unit='h')

    # 외기 조건 (계절성 및 일일 변화 포함)
    data['outdoor_temp_c'] = 15 + 10 * np.sin(2 * np.pi * data.index / (24 * 30)) + 5 * np.sin(2 * np.pi * data.index / 24) + np.random.normal(0, 1, num_rows)
    data['outdoor_rh_percent'] = 60 + 15 * np.sin(2 * np.pi * data.index / (24*15)) + 10 * np.sin(2*np.pi*data.index/24 + np.pi/2) + np.random.normal(0,5,num_rows)
    data['outdoor_rh_percent'] = np.clip(data['outdoor_rh_percent'], 20, 95)

    # OAC 코일 설정 및 실제 값 (단일 유닛 기준 예시)
    # 실제로는 이 값들이 예측 대상이거나, 복잡한 상호작용의 결과입니다.
    # 여기서는 학습을 위해 임의의 관계를 가진 더미 값을 생성합니다.
    coil_types = ['preheating', 'precooling', 'cooling', 'reheating']
    current_temp = data['outdoor_temp_c'].copy()
    current_rh = data['outdoor_rh_percent'].copy()

    enthalpy_calc = EnthalpyCalculator() # 임시 사용

    for coil in coil_types:
        data[f'{coil}_set_point_temp_c'] = np.random.uniform(10, 25, num_rows)
        if coil == 'preheating':
            data[f'{coil}_set_point_temp_c'] = np.where(current_temp < 18, 20, current_temp)
        elif coil == 'precooling':
             data[f'{coil}_set_point_temp_c'] = np.where(current_temp > 25, 22, current_temp)
        elif coil == 'cooling':
            data[f'{coil}_set_point_temp_c'] = 12.0
        elif coil == 'reheating':
             data[f'{coil}_set_point_temp_c'] = 22.0

        # 매우 단순화된 개도율 로직 (학습 타겟용)
        delta_temp_target = data[f'{coil}_set_point_temp_c'] - current_temp
        if coil in ['preheating', 'reheating']: # 가열 코일
            data[f'{coil}_coil_open_rate'] = np.clip(delta_temp_target / 10.0, 0, 1) # 10도 차이에 100% 개도 가정
        else: # 냉각 코일
            data[f'{coil}_coil_open_rate'] = np.clip(-delta_temp_target / 10.0, 0, 1)

        data[f'{coil}_coil_open_rate'] = data[f'{coil}_coil_open_rate'] * np.random.uniform(0.7, 1.3, num_rows) # Noise
        data[f'{coil}_coil_open_rate'] = np.clip(data[f'{coil}_coil_open_rate'], 0, 1)

        # 코일 통과 후 온도/습도 (매우 단순화)
        # 실제로는 엔탈피 기반으로 더 정교하게 계산되어야 함.
        # design_delta_h 값을 사용하여 outlet 상태를 계산해야 OACModel의 예측과 일관성 유지
        # 여기서는 OACModel의 design_delta_h를 가져올 수 없으므로, 근사치 사용
        design_delta_h_approx = {'preheating': 15000, 'precooling': -20000, 'cooling': -25000, 'reheating': 12000}

        inlet_h = enthalpy_calc.calculate_enthalpy(current_temp, current_rh)
        sim_delta_h = design_delta_h_approx[coil] * data[f'{coil}_coil_open_rate']
        outlet_h = inlet_h + sim_delta_h

        # outlet_temp, outlet_rh는 enthalpy_calc.calculate_temp_humidity_from_enthalpy로 계산해야 하나,
        # 해당 함수가 부정확하므로, 여기서는 set_point에 가깝게 변한다고 단순화.
        data[f'{coil}_outlet_temp_c'] = current_temp + (data[f'{coil}_set_point_temp_c'] - current_temp) * data[f'{coil}_coil_open_rate'] * 0.8 # 80% 효율 가정
        data[f'{coil}_outlet_rh_percent'] = current_rh # 습도 변화는 일단 무시 (단순화)

        current_temp = data[f'{coil}_outlet_temp_c'].copy()
        # current_rh = data[f'{coil}_outlet_rh_percent'].copy() # 실제로는 습도도 변함

    data['humidification_set_point_rate'] = np.random.uniform(0, 0.3, num_rows) # 가습은 보통 겨울에만
    data['humidification_coil_open_rate'] = data['humidification_set_point_rate'] * np.random.uniform(0.8,1.2,num_rows)
    data['humidification_coil_open_rate'] = np.clip(data['humidification_coil_open_rate'],0,1)

    # 냉동기 부하 (OAC 코일 부하로부터 계산된다고 가정)
    # air_mass_flow_rate_per_oac_unit_kg_s = 12.0 (HVACSystemFacade에서 가져와야 하나, 여기서는 상수 사용)
    # num_oac_units = 14
    # delta_h_precooling = design_delta_h_approx['precooling'] * data['precooling_coil_open_rate']
    # delta_h_cooling = design_delta_h_approx['cooling'] * data['cooling_coil_open_rate']
    # data['total_precooling_load_watts'] = abs(delta_h_precooling * 12.0 * 14)
    # data['total_cooling_load_watts'] = abs(delta_h_cooling * 12.0 * 14)
    # 위 방식 대신, 부하를 임의로 생성 (모델 학습용이므로)
    data['total_precooling_load_watts'] = np.random.uniform(100000, 500000, num_rows) * data['precooling_coil_open_rate']
    data['total_cooling_load_watts'] = np.random.uniform(200000, 800000, num_rows) * data['cooling_coil_open_rate']


    # 냉동기 운전 상태 (학습 타겟용)
    for temp_type in ["low_temp", "high_temp"]:
        load_key = 'total_cooling_load_watts' if temp_type == "low_temp" else 'total_precooling_load_watts'
        # 가동 대수는 부하에 비례한다고 가정 (단순화)
        data[f'{temp_type}_chiller_active_count'] = np.ceil(data[load_key] / 200000).astype(int) # 1대당 200kW 용량 가정
        data[f'{temp_type}_chiller_active_count'] = np.clip(data[f'{temp_type}_chiller_active_count'], 1, 8)

        data[f'{temp_type}_main_pressure_bar'] = 1.5 + np.random.normal(0, 0.1, num_rows) - (data[f'{temp_type}_chiller_active_count']-4)*0.05
        data[f'{temp_type}_sub_pressure_valve_rate_percent'] = 60 + np.random.normal(0,5,num_rows) + data[load_key]/50000 - (data[f'{temp_type}_chiller_active_count']-4)*5
        data[f'{temp_type}_main_control_valve_rate_percent'] = 70 + np.random.normal(0,5,num_rows) + data[load_key]/60000 - (data[f'{temp_type}_chiller_active_count']-4)*5

        data[f'{temp_type}_main_pressure_bar'] = np.clip(data[f'{temp_type}_main_pressure_bar'], 1.0, 2.0)
        data[f'{temp_type}_sub_pressure_valve_rate_percent'] = np.clip(data[f'{temp_type}_sub_pressure_valve_rate_percent'], 10, 100)
        data[f'{temp_type}_main_control_valve_rate_percent'] = np.clip(data[f'{temp_type}_main_control_valve_rate_percent'], 10, 100)

    return data

class EnthalpyCalculator:
    def calculate_enthalpy(self, temperature_c, relative_humidity_percent, pressure_pa=101325):
        # 매우 단순화된 placeholder 유지
        return (temperature_c * 1000 + (relative_humidity_percent / 100) * 2500 * 10) * (1 + (temperature_c / 100))

    def calculate_temp_humidity_from_enthalpy(self, enthalpy_j_kg, initial_temp_c, pressure_pa=101325):
        initial_h_approx = self.calculate_enthalpy(initial_temp_c, 50)
        delta_h = enthalpy_j_kg - initial_h_approx
        delta_temp_c = delta_h / (1005 + 1860 * 0.01)
        new_temp_c = initial_temp_c + (delta_temp_c * 0.2)
        humidity_change_factor = (enthalpy_j_kg - initial_h_approx) / 10000.0
        new_humidity_percent = 50 + humidity_change_factor * 10
        new_humidity_percent = min(max(new_humidity_percent, 5.0), 99.0)
        new_temp_c = min(max(new_temp_c, -10.0), 50.0)
        return new_temp_c, new_humidity_percent

class OACModel:
    def __init__(self, num_oac_units=14):
        self.num_oac_units = num_oac_units
        self.enthalpy_calculator = EnthalpyCalculator()
        self.coil_order = ['preheating', 'precooling', 'cooling', 'reheating', 'humidification']

        self.trained_models = {} # 학습된 모델 저장용
        self.design_delta_h = {
            'preheating': 15000, 'precooling': -20000, 'cooling': -25000,
            'reheating': 12000, 'humidification': 8000
        }

    def train_coil_models(self, df_train_data):
        """각 OAC 코일의 개도율 예측 모델을 학습합니다."""
        print("\n--- OAC 코일 모델 학습 시작 ---")

        # 외기 조건 엔탈피 계산
        df_train_data['outdoor_enthalpy'] = self.enthalpy_calculator.calculate_enthalpy(
            df_train_data['outdoor_temp_c'], df_train_data['outdoor_rh_percent']
        )

        # inlet_temp_col = 'outdoor_temp_c' # 초기 입구는 외기
        # inlet_rh_col = 'outdoor_rh_percent' # 초기 입구는 외기
        # inlet_h_col = 'outdoor_enthalpy' # 초기 입구는 외기

        for coil_type in self.coil_order:
            print(f"  {coil_type} 코일 모델 학습 중...")

            target_col = f'{coil_type}_coil_open_rate'
            if target_col not in df_train_data.columns:
                print(f"    경고: {target_col}이 학습 데이터에 없어 {coil_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                continue

            # 특징(Features) 구성:
            # 기본적으로 외기 조건(온도, 습도, 엔탈피)을 포함.
            # 예열 코일 이후부터는 이전 코일의 실제 출구 상태(온도, 습도, 엔탈피)를 특징으로 추가.
            # 각 코일의 설정값(목표 온도 또는 가습률)을 특징으로 추가.

            temp_df_train = df_train_data.copy() # 데이터 복사하여 사용

            if coil_type == 'preheating':
                current_features = ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy',
                                    f'{coil_type}_set_point_temp_c']
            elif coil_type == 'humidification':
                prev_coil = self.coil_order[self.coil_order.index(coil_type)-1]
                # 이전 코일의 실제 출구 상태 컬럼명 (더미 데이터 생성 시 이름 규칙에 맞게)
                prev_outlet_temp_col = f'{prev_coil}_outlet_temp_c'
                prev_outlet_rh_col = f'{prev_coil}_outlet_rh_percent'

                # 이전 코일 출구 엔탈피 계산
                temp_df_train[f'{prev_coil}_outlet_h_actual'] = self.enthalpy_calculator.calculate_enthalpy(
                    temp_df_train[prev_outlet_temp_col], temp_df_train[prev_outlet_rh_col]
                )
                current_features = ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy', # 외기 조건은 항상 포함
                                   prev_outlet_temp_col, prev_outlet_rh_col, f'{prev_coil}_outlet_h_actual', # 이전 코일 출구
                                   'humidification_set_point_rate'] # 가습 설정률
            else: # precooling, cooling, reheating
                prev_coil = self.coil_order[self.coil_order.index(coil_type)-1]
                prev_outlet_temp_col = f'{prev_coil}_outlet_temp_c'
                prev_outlet_rh_col = f'{prev_coil}_outlet_rh_percent'

                temp_df_train[f'{prev_coil}_outlet_h_actual'] = self.enthalpy_calculator.calculate_enthalpy(
                    temp_df_train[prev_outlet_temp_col], temp_df_train[prev_outlet_rh_col]
                )
                current_features = ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy',
                                   prev_outlet_temp_col, prev_outlet_rh_col, f'{prev_coil}_outlet_h_actual',
                                   f'{coil_type}_set_point_temp_c'] # 현재 코일 목표 온도

            X = temp_df_train[current_features].fillna(method='ffill').fillna(method='bfill') # 결측치 처리
            y = temp_df_train[target_col]

            if X.empty or y.empty or X.shape[0] != y.shape[0]:
                print(f"    오류: 특징 또는 타겟 데이터가 비어있거나 크기가 맞지 않아 {coil_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                continue

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1, max_depth=10, min_samples_split=10, min_samples_leaf=5) # 간소화된 파라미터
            model.fit(X_train, y_train)

            y_pred_test = model.predict(X_test)
            mse_test = mean_squared_error(y_test, y_pred_test)
            print(f"    {coil_type} 모델 학습 완료. Test MSE: {mse_test:.4f}")
            self.trained_models[coil_type] = model
            # joblib.dump(model, f'{coil_type}_coil_model.pkl') # 실제 파일 저장

        print("--- OAC 코일 모델 학습 완료 ---")


    def predict_coil_open_rates(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points, modified_coil_states=None):
        if not self.trained_models:
            # print("경고: OAC 모델이 학습되지 않았습니다. 규칙 기반으로 예측합니다.")
            return self._predict_coil_open_rates_rule_based(outdoor_temp_c, outdoor_rh_percent, oac_set_points, modified_coil_states)

        if modified_coil_states is None: modified_coil_states = {}

        current_temp_c = outdoor_temp_c
        current_rh_percent = outdoor_rh_percent
        current_h_j_kg = self.enthalpy_calculator.calculate_enthalpy(current_temp_c, current_rh_percent)

        outdoor_enthalpy_val = current_h_j_kg # 초기 외기 엔탈피 (예측 시 사용)
        results = {}

        # 이전 코일의 실제 출구 상태를 저장할 변수 (예측 시 다음 코일의 입력으로 사용)
        prev_coil_actual_outlet_temp = outdoor_temp_c
        prev_coil_actual_outlet_rh = outdoor_rh_percent
        # prev_coil_actual_outlet_h = outdoor_enthalpy_val # 아래 루프에서 계산됨

        for coil_type in self.coil_order:
            inlet_temp_c = current_temp_c
            inlet_rh_percent = current_rh_percent
            inlet_h_j_kg = current_h_j_kg

            predicted_open_rate = 0.0
            # actual_delta_h_j_kg = 0.0 # 아래에서 계산

            if coil_type in modified_coil_states:
                predicted_open_rate = modified_coil_states[coil_type]
            elif coil_type in self.trained_models:
                model = self.trained_models[coil_type]

                # 모델 입력 특징(features_values) 준비
                # 학습 시 사용된 current_features 리스트의 순서와 동일하게 값을 준비해야 함.
                if coil_type == 'preheating':
                    # features: ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy', 'preheating_set_point_temp_c']
                    feature_values_for_pred = [outdoor_temp_c, outdoor_rh_percent, outdoor_enthalpy_val,
                                               oac_set_points.get(coil_type, current_temp_c)]
                    # 학습 시 사용된 컬럼명으로 DataFrame 생성
                    df_predict_columns = ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy', f'{coil_type}_set_point_temp_c']

                elif coil_type == 'humidification':
                    # features: ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy',
                    #            prev_outlet_temp_col, prev_outlet_rh_col, f'{prev_coil}_outlet_h_actual',
                    #            'humidification_set_point_rate']
                    prev_coil_h_actual = self.enthalpy_calculator.calculate_enthalpy(prev_coil_actual_outlet_temp, prev_coil_actual_outlet_rh)
                    feature_values_for_pred = [outdoor_temp_c, outdoor_rh_percent, outdoor_enthalpy_val,
                                               prev_coil_actual_outlet_temp, prev_coil_actual_outlet_rh, prev_coil_h_actual,
                                               oac_set_points.get('humidification_rate', 0.0)]
                    prev_coil_name_for_cols = self.coil_order[self.coil_order.index(coil_type)-1]
                    df_predict_columns = ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy',
                                          f'{prev_coil_name_for_cols}_outlet_temp_c', f'{prev_coil_name_for_cols}_outlet_rh_percent', f'{prev_coil_name_for_cols}_outlet_h_actual',
                                          'humidification_set_point_rate']
                else: # precooling, cooling, reheating
                    # features: ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy',
                    #            prev_outlet_temp_col, prev_outlet_rh_col, f'{prev_coil}_outlet_h_actual',
                    #            f'{coil_type}_set_point_temp_c']
                    prev_coil_h_actual = self.enthalpy_calculator.calculate_enthalpy(prev_coil_actual_outlet_temp, prev_coil_actual_outlet_rh)
                    feature_values_for_pred = [outdoor_temp_c, outdoor_rh_percent, outdoor_enthalpy_val,
                                               prev_coil_actual_outlet_temp, prev_coil_actual_outlet_rh, prev_coil_h_actual,
                                               oac_set_points.get(coil_type, current_temp_c)]
                    prev_coil_name_for_cols = self.coil_order[self.coil_order.index(coil_type)-1]
                    df_predict_columns = ['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy',
                                          f'{prev_coil_name_for_cols}_outlet_temp_c', f'{prev_coil_name_for_cols}_outlet_rh_percent', f'{prev_coil_name_for_cols}_outlet_h_actual',
                                          f'{coil_type}_set_point_temp_c']
                try:
                    input_df = pd.DataFrame([feature_values_for_pred], columns=df_predict_columns)
                    predicted_open_rate = model.predict(input_df)[0]
                    predicted_open_rate = min(max(predicted_open_rate, 0.0), 1.0) # 0~1 클리핑
                except Exception as e:
                    print(f"    오류: {coil_type} 모델 예측 중 오류 발생 - {e}. 개도율 0으로 설정.")
                    predicted_open_rate = 0.0
            else:
                 predicted_open_rate = 0.0

            actual_delta_h_j_kg = self.design_delta_h[coil_type] * predicted_open_rate
            outlet_h_j_kg = inlet_h_j_kg + actual_delta_h_j_kg
            current_temp_c, current_rh_percent = self.enthalpy_calculator.calculate_temp_humidity_from_enthalpy(outlet_h_j_kg, inlet_temp_c)
            current_h_j_kg = outlet_h_j_kg # 엔탈피 계산 정확도에 따라 이 값은 재계산 필요할 수 있음

            results[coil_type] = {
                'open_rate': predicted_open_rate,
                'inlet_temp_c': inlet_temp_c, 'inlet_rh_percent': inlet_rh_percent, 'inlet_h_j_kg': inlet_h_j_kg,
                'outlet_temp_c': current_temp_c, 'outlet_rh_percent': current_rh_percent, 'outlet_h_j_kg': current_h_j_kg,
                'delta_h_j_kg': actual_delta_h_j_kg,
                'target_outlet_temp_c': oac_set_points.get(coil_type, 'N/A') if coil_type != 'humidification' else oac_set_points.get('humidification_rate','N/A')
            }

            # 현재 코일의 실제 출구 상태를 다음 코일 예측을 위해 업데이트
            prev_coil_actual_outlet_temp = current_temp_c
            prev_coil_actual_outlet_rh = current_rh_percent
            # prev_coil_actual_outlet_h 은 다음 루프 시작 시 prev_coil_h_actual로 계산됨.

        results['final_outlet_temp_c'] = current_temp_c
        results['final_outlet_rh_percent'] = current_rh_percent
        results['final_outlet_h_j_kg'] = current_h_j_kg
        return results

    def _predict_coil_open_rates_rule_based(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points, modified_coil_states=None):
        # 이 함수는 모델이 없을 때 fallback으로 사용됩니다. (OACModel의 초기 버전 predict_coil_open_rates 내용)
        if modified_coil_states is None: modified_coil_states = {}
        current_temp_c = outdoor_temp_c
        current_rh_percent = outdoor_rh_percent
        current_h_j_kg = self.enthalpy_calculator.calculate_enthalpy(current_temp_c, current_rh_percent)
        results = {}
        for coil_type in self.coil_order:
            inlet_temp_c, inlet_rh_percent, inlet_h_j_kg = current_temp_c, current_rh_percent, current_h_j_kg
            target_outlet_temp_c = oac_set_points.get(coil_type)
            predicted_open_rate, actual_delta_h_j_kg = 0.0, 0.0

            if coil_type in modified_coil_states:
                predicted_open_rate = modified_coil_states[coil_type]
                actual_delta_h_j_kg = self.design_delta_h[coil_type] * predicted_open_rate
            elif coil_type == 'humidification':
                predicted_open_rate = oac_set_points.get('humidification_rate', 0.0)
                actual_delta_h_j_kg = self.design_delta_h[coil_type] * predicted_open_rate
            elif target_outlet_temp_c is not None:
                # 목표 온도를 기준으로 엔탈피 변화량 계산 (규칙 기반)
                target_outlet_h_j_kg = self.enthalpy_calculator.calculate_enthalpy(target_outlet_temp_c, current_rh_percent) # 목표 습도는 현재와 동일 가정
                required_delta_h_j_kg = target_outlet_h_j_kg - inlet_h_j_kg
                if self.design_delta_h[coil_type] > 0: # 가열 코일
                    predicted_open_rate = required_delta_h_j_kg / self.design_delta_h[coil_type] if required_delta_h_j_kg > 0 and self.design_delta_h[coil_type] !=0 else 0.0
                elif self.design_delta_h[coil_type] < 0: # 냉각 코일
                    predicted_open_rate = required_delta_h_j_kg / self.design_delta_h[coil_type] if required_delta_h_j_kg < 0 and self.design_delta_h[coil_type] !=0 else 0.0
                else: # design_delta_h가 0인 경우 (오류 방지)
                    predicted_open_rate = 0.0
                predicted_open_rate = min(max(predicted_open_rate, 0.0), 1.0)
                actual_delta_h_j_kg = self.design_delta_h[coil_type] * predicted_open_rate

            outlet_h_j_kg = inlet_h_j_kg + actual_delta_h_j_kg
            current_temp_c, current_rh_percent = self.enthalpy_calculator.calculate_temp_humidity_from_enthalpy(outlet_h_j_kg, inlet_temp_c)
            current_h_j_kg = outlet_h_j_kg
            results[coil_type] = {
                'open_rate': predicted_open_rate, 'inlet_temp_c': inlet_temp_c, 'inlet_rh_percent': inlet_rh_percent,
                'inlet_h_j_kg': inlet_h_j_kg, 'outlet_temp_c': current_temp_c, 'outlet_rh_percent': current_rh_percent,
                'outlet_h_j_kg': current_h_j_kg, 'delta_h_j_kg': actual_delta_h_j_kg,
                'target_outlet_temp_c': target_outlet_temp_c if coil_type != 'humidification' else oac_set_points.get('humidification_rate','N/A')
            }
        results['final_outlet_temp_c'], results['final_outlet_rh_percent'], results['final_outlet_h_j_kg'] = current_temp_c, current_rh_percent, current_h_j_kg
        return results


class ChillerModel:
    def __init__(self):
        self.trained_models = {} # 학습된 모델 저장용 (low_temp, high_temp)

    def train_pressure_valve_models(self, df_train_data):
        """냉동기 차압 및 밸브 개도율 예측 모델을 학습합니다."""
        print("\n--- 냉동기 차압/밸브 모델 학습 시작 ---")
        for chiller_type in ["low_temp", "high_temp"]:
            print(f"  {chiller_type} 냉동기 모델 학습 중...")

            load_col = 'total_cooling_load_watts' if chiller_type == "low_temp" else 'total_precooling_load_watts'
            active_count_col = f'{chiller_type}_chiller_active_count'

            features_cols = [load_col, active_count_col] # 학습에 사용될 특징 컬럼명
            target_cols = [f'{chiller_type}_main_pressure_bar',
                           f'{chiller_type}_sub_pressure_valve_rate_percent',
                           f'{chiller_type}_main_control_valve_rate_percent']

            if not all(col in df_train_data.columns for col in features_cols):
                print(f"    경고: 특징 컬럼({features_cols})이 학습 데이터에 없어 {chiller_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                continue
            if not all(col in df_train_data.columns for col in target_cols):
                print(f"    경고: 타겟 컬럼 중 일부가 학습 데이터에 없어 {chiller_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                continue


            X = df_train_data[features_cols].fillna(method='ffill').fillna(method='bfill')

            type_models = {} # 해당 타입 냉동기의 타겟별 모델 저장
            for target_col in target_cols:
                y = df_train_data[target_col]
                if X.empty or y.empty or X.shape[0] != y.shape[0]:
                    print(f"    오류: 특징 또는 타겟({target_col}) 데이터가 비어있거나 크기가 맞지 않아 학습을 건너<0xEB><0><0x84>니다.")
                    continue

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                model = RandomForestRegressor(n_estimators=30, random_state=42, n_jobs=-1, max_depth=8, min_samples_split=8,min_samples_leaf=4) # 간소화
                model.fit(X_train, y_train)

                y_pred_test = model.predict(X_test)
                mse_test = mean_squared_error(y_test, y_pred_test)
                print(f"    {target_col} 모델 학습 완료. Test MSE: {mse_test:.4f}")
                type_models[target_col] = model # 타겟 컬럼명을 key로 모델 저장
                # joblib.dump(model, f'{target_col}_model.pkl')
            self.trained_models[chiller_type] = type_models # 냉동기 타입을 key로 모델 딕셔너리 저장
        print("--- 냉동기 차압/밸브 모델 학습 완료 ---")

    def predict_pressure_valve(self, cooling_load_watts, num_active_chillers, chiller_type="low_temp"):
        if chiller_type not in self.trained_models or not self.trained_models[chiller_type]:
            # print(f"경고: {chiller_type} 냉동기 모델이 학습되지 않았습니다. 기본값을 반환합니다.")
            return {"main_pressure_bar": 1.5, "sub_pressure_valve_rate_percent": 60.0, "main_control_valve_rate_percent": 70.0}

        models_for_type = self.trained_models[chiller_type] # 해당 타입 냉동기의 모델 딕셔너리

        # 예측 시 입력 DataFrame의 컬럼명은 학습 시 사용한 features_cols와 일치해야 함
        load_col_name_at_train = 'total_cooling_load_watts' if chiller_type == "low_temp" else 'total_precooling_load_watts'
        active_count_col_name_at_train = f'{chiller_type}_chiller_active_count'

        input_df_for_pred = pd.DataFrame([[cooling_load_watts, num_active_chillers]],
                                         columns=[load_col_name_at_train, active_count_col_name_at_train])

        predictions = {}
        # 학습된 모델 딕셔너리(models_for_type)의 key는 타겟 컬럼명임
        for target_col_name_at_train, model in models_for_type.items():
            try:
                pred_val = model.predict(input_df_for_pred)[0]
                # 출력 딕셔너리의 key는 'main_pressure_bar' 등 일반적인 이름으로 변환
                output_key = target_col_name_at_train.replace(f'{chiller_type}_', '')
                predictions[output_key] = pred_val
            except Exception as e:
                output_key = target_col_name_at_train.replace(f'{chiller_type}_', '')
                print(f"    오류: {output_key} ({target_col_name_at_train}) 예측 중 오류 - {e}. 기본값 사용.")
                if "pressure_bar" in output_key: predictions[output_key] = 1.5
                elif "sub_pressure_valve_rate_percent" in output_key: predictions[output_key] = 60.0
                elif "main_control_valve_rate_percent" in output_key: predictions[output_key] = 70.0
                else: predictions[output_key] = 0 # 기타 알 수 없는 경우

        # 모든 기대 키가 채워졌는지 확인
        expected_output_keys = ["main_pressure_bar", "sub_pressure_valve_rate_percent", "main_control_valve_rate_percent"]
        for key in expected_output_keys:
            if key not in predictions:
                print(f"    경고: 예측 결과에 {key}가 누락되어 기본값으로 채웁니다.")
                if "pressure" in key: predictions[key] = 1.5
                elif "sub" in key: predictions[key] = 60.0
                else: predictions[key] = 70.0

        return predictions


class ChillerOptimizer:
    def __init__(self, num_low_temp_chillers=8, num_high_temp_chillers=8):
        self.num_low_temp_chillers = num_low_temp_chillers
        self.num_high_temp_chillers = num_high_temp_chillers
        self.chiller_specs = {
            "low_temp": [{"id": i, "max_capacity_watts": 200000, "cop_points": {0.3: 3.0, 0.55: 3.8, 0.8: 3.5, 1.0: 3.0}} for i in range(num_low_temp_chillers)],
            "high_temp": [{"id": i, "max_capacity_watts": 250000, "cop_points": {0.3: 3.2, 0.55: 4.0, 0.8: 3.7, 1.0: 3.2}} for i in range(num_high_temp_chillers)]
        }
        for type_key in self.chiller_specs: # COP 좋은 순으로 정렬
            self.chiller_specs[type_key] = sorted(self.chiller_specs[type_key], key=lambda x: self._get_cop_at_load(x['cop_points'], 0.55), reverse=True)
        self.target_load_percentage = 0.55
        self.min_load_percentage = 0.30
        self.max_load_percentage = 0.90

    def _get_cop_at_load(self, cop_points, load_percentage):
        if not cop_points: return 3.0 # 기본 COP
        load_percentage = min(max(load_percentage, 0.0), 1.0) # 부하율은 0~1 사이
        if load_percentage == 0: return 0 # 부하 0이면 COP 정의 어려움 (전력소모 0 가정)

        sorted_load_points = sorted(cop_points.keys())

        if load_percentage <= sorted_load_points[0]: return cop_points[sorted_load_points[0]]
        if load_percentage >= sorted_load_points[-1]: return cop_points[sorted_load_points[-1]]

        for i in range(len(sorted_load_points) - 1):
            x1, y1 = sorted_load_points[i], cop_points[sorted_load_points[i]]
            x2, y2 = sorted_load_points[i+1], cop_points[sorted_load_points[i+1]]
            if x1 <= load_percentage < x2:
                # 선형 보간
                return y1 + (y2 - y1) * (load_percentage - x1) / (x2 - x1) if (x2-x1) != 0 else y1
        return cop_points[sorted_load_points[-1]] # 마지막 포인트 COP 반환

    def optimize_chiller_operation(self, required_total_load_watts, chiller_type="low_temp"):
        specs_list = self.chiller_specs[chiller_type]
        num_total_chillers = len(specs_list)

        best_config = {'num_active': 0, 'active_chiller_ids': [], 'load_percentages': [], 'total_power': float('inf'), 'avg_load_percentage': 0, 'meets_target_load_rate': False}

        if required_total_load_watts <= 0: # 부하가 없으면 가동 불필요
            return 0, [], [], 0, 0, True # 평균 부하율 0, 목표 달성으로 간주

        # 1대부터 N대까지 가동하는 모든 경우를 탐색
        for num_active in range(1, num_total_chillers + 1):
            current_chiller_selection_specs = specs_list[:num_active] # COP 좋은 순으로 선택

            # 선택된 냉동기들로 부하 감당 가능한지 확인
            total_max_cap_selected = sum(s['max_capacity_watts'] for s in current_chiller_selection_specs)
            if total_max_cap_selected * self.max_load_percentage < required_total_load_watts:
                continue # 최대 부하율로도 감당 안되면 다음 가동 대수 검토
            # 최소 부하율 조건도 고려 (부하가 너무 작아도 비효율적)
            if total_max_cap_selected * self.min_load_percentage > required_total_load_watts and required_total_load_watts > 0:
                if num_active > 1 : # 1대일때는 이 조건 무시 (최소 1대는 켜야 하므로)
                    continue


            # 부하 균등 분배 시도
            load_per_chiller_target = required_total_load_watts / num_active
            current_load_percentages = []
            current_total_power = 0
            possible_combination = True
            active_ids_temp = []

            for chiller_spec in current_chiller_selection_specs:
                active_ids_temp.append(chiller_spec['id'])
                load_percentage_this_chiller = load_per_chiller_target / chiller_spec['max_capacity_watts']

                # 개별 냉동기 부하율 제약 확인
                if not (self.min_load_percentage <= load_percentage_this_chiller <= self.max_load_percentage):
                    possible_combination = False
                    break

                current_load_percentages.append(load_percentage_this_chiller)
                cop = self._get_cop_at_load(chiller_spec['cop_points'], load_percentage_this_chiller)
                if cop <= 0: # COP가 0이거나 음수면 계산 불가
                    possible_combination = False; break
                power_consumption_this_chiller = (chiller_spec['max_capacity_watts'] * load_percentage_this_chiller) / cop
                current_total_power += power_consumption_this_chiller

            if not possible_combination:
                continue

            avg_load_percentage = sum(current_load_percentages) / num_active if num_active > 0 else 0

            meets_target = abs(avg_load_percentage - self.target_load_percentage) < 0.05 # 5% 이내면 목표 달성

            is_better = False
            if not best_config['active_chiller_ids']: # 첫 유효 조합
                is_better = True
            elif best_config['meets_target_load_rate'] and meets_target: # 둘 다 목표 부하율 만족
                if current_total_power < best_config['total_power']: is_better = True # 전력 적은 쪽
            elif meets_target and not best_config['meets_target_load_rate']: # 현재만 목표 만족
                is_better = True
            elif not best_config['meets_target_load_rate'] and not meets_target: # 둘 다 목표 미만족
                # 목표 부하율에 더 가깝거나, 같다면 전력 소비가 적은 것
                if abs(avg_load_percentage - self.target_load_percentage) < abs(best_config['avg_load_percentage'] - self.target_load_percentage):
                    is_better = True
                elif abs(avg_load_percentage - self.target_load_percentage) == abs(best_config['avg_load_percentage'] - self.target_load_percentage):
                    if current_total_power < best_config['total_power']: is_better = True

            if is_better:
                best_config['num_active'] = num_active
                best_config['active_chiller_ids'] = sorted(active_ids_temp)
                best_config['load_percentages'] = current_load_percentages
                best_config['total_power'] = current_total_power
                best_config['avg_load_percentage'] = avg_load_percentage
                best_config['meets_target_load_rate'] = meets_target

        if not best_config['active_chiller_ids'] and required_total_load_watts > 0 :
            # 위에서 적절한 조합을 찾지 못한 경우 (fallback: 모든 냉동기 가동)
            # print(f"  경고: {chiller_type} 최적 조합 탐색 실패. Fallback 로직 적용 (모든 냉동기 가동 시도).")
            num_active = num_total_chillers
            active_ids_temp = [s['id'] for s in specs_list]
            load_per_chiller = required_total_load_watts / num_active if num_active > 0 else 0
            current_load_percentages = []
            current_total_power = 0
            for chiller_spec in specs_list: # 모든 냉동기 사용
                load_percentage = load_per_chiller / chiller_spec['max_capacity_watts'] if chiller_spec['max_capacity_watts'] > 0 else 0
                # 부하율이 1을 넘거나 min_load_percentage보다 작을 수 있음. 실제 운전 불가 상황.
                # 여기서는 COP 계산을 위해 부하율을 min/max_load_percentage 사이로 클리핑하여 COP 계산.
                # 하지만 실제 부하량은 클리핑 전 값으로 계산.
                effective_load_perc_for_cop = min(max(load_percentage, self.min_load_percentage), self.max_load_percentage)

                current_load_percentages.append(load_percentage) # 실제 부하율 기록
                cop = self._get_cop_at_load(chiller_spec['cop_points'], effective_load_perc_for_cop)
                if cop <= 0: cop = 1.0 # 비상시 COP
                current_total_power += (chiller_spec['max_capacity_watts'] * load_percentage) / cop # 실제 부하량으로 전력 계산

            best_config['num_active'] = num_active
            best_config['active_chiller_ids'] = sorted(active_ids_temp)
            best_config['load_percentages'] = current_load_percentages
            best_config['total_power'] = current_total_power
            best_config['avg_load_percentage'] = sum(lp for lp in current_load_percentages if lp <=1.0) / num_active if num_active > 0 else 0 # 100% 초과 부하율은 평균 계산 시 왜곡 가능성
            best_config['meets_target_load_rate'] = abs(best_config['avg_load_percentage'] - self.target_load_percentage) < 0.10 # fallback은 기준 완화

        return best_config['num_active'], best_config['active_chiller_ids'], best_config['load_percentages'], best_config['total_power'], best_config['avg_load_percentage'], best_config['meets_target_load_rate']

class CoolingTowerOptimizer:
    def __init__(self, num_towers=9):
        self.tower_specs = [
            {"id": 0, "max_capacity_watts": 300000, "fan_power_watts": 7000},
            {"id": 1, "max_capacity_watts": 300000, "fan_power_watts": 7000},
            {"id": 2, "max_capacity_watts": 350000, "fan_power_watts": 8000},
            {"id": 3, "max_capacity_watts": 320000, "fan_power_watts": 7500},
            {"id": 4, "max_capacity_watts": 320000, "fan_power_watts": 7500}
        ] + [{"id": i, "max_capacity_watts": 330000, "fan_power_watts": 7700} for i in range(5, num_towers)]
        self.num_towers = len(self.tower_specs)
        # 효율 좋은 순 (단위 용량당 팬 전력 낮은 순)으로 정렬
        self.sorted_towers = sorted(self.tower_specs, key=lambda x: x['fan_power_watts'] / x['max_capacity_watts'] if x['max_capacity_watts'] > 0 else float('inf'))

    def optimize_tower_operation(self, total_heat_rejection_load_watts):
        active_tower_ids = []
        current_total_capacity = 0
        current_total_fan_power = 0

        if total_heat_rejection_load_watts <= 0:
            return [], 0, 0, 0

        # Greedy 접근: 효율 좋은 타워부터 순서대로 켜면서 필요 용량 만족하는지 확인
        for tower in self.sorted_towers:
            if current_total_capacity >= total_heat_rejection_load_watts:
                break # 이미 필요 용량 충족

            active_tower_ids.append(tower['id'])
            current_total_capacity += tower['max_capacity_watts']
            current_total_fan_power += tower['fan_power_watts']

        # if current_total_capacity < total_heat_rejection_load_watts:
            # print(f"  경고: 모든 냉각탑 가동({len(active_tower_ids)}대)으로도 총 방열부하({total_heat_rejection_load_watts:,.0f}W)를 만족하지 못할 수 있습니다. (확보 용량: {current_total_capacity:,.0f}W)")
            # pass # 모든 타워를 켜도 부족한 경우, 일단 모두 켠 상태로 결과 반환

        return sorted(active_tower_ids), current_total_fan_power, current_total_capacity, len(active_tower_ids)

class HVACSystemFacade:
    def __init__(self, oac_model_instance, chiller_model_instance): # 학습된 모델 인스턴스 주입
        self.oac_model = oac_model_instance
        self.chiller_model = chiller_model_instance
        self.chiller_optimizer = ChillerOptimizer()
        self.cooling_tower_optimizer = CoolingTowerOptimizer()
        self.air_mass_flow_rate_per_oac_unit_kg_s = 12.0 # OAC 유닛당 공기 질량 유량 (kg/s)
        self.num_oac_units = 14 # 총 OAC 대수

    def predict_and_optimize(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points,
                             chilled_water_supply_set_temp, modified_oac_coil_states=None):
        # 1. OAC 코일 개도율 및 각 단 출구 상태 예측
        oac_results_single_unit = self.oac_model.predict_coil_open_rates(
            outdoor_temp_c, outdoor_rh_percent, oac_set_points, modified_oac_coil_states
        )

        # 2. 냉동기 부하 계산 (전체 OAC 유닛 대상)
        delta_h_precooling_j_kg = oac_results_single_unit.get('precooling', {}).get('delta_h_j_kg', 0)
        total_precooling_load_watts = abs(delta_h_precooling_j_kg * self.air_mass_flow_rate_per_oac_unit_kg_s * self.num_oac_units)

        delta_h_cooling_j_kg = oac_results_single_unit.get('cooling', {}).get('delta_h_j_kg', 0)
        total_cooling_load_watts = abs(delta_h_cooling_j_kg * self.air_mass_flow_rate_per_oac_unit_kg_s * self.num_oac_units)

        # 3. 저온 냉동기 최적화 및 예측 (주 냉각 코일 담당)
        optim_low_ch_count, active_low_ids, load_dist_low, power_low, avg_load_low, meets_target_low = \
            self.chiller_optimizer.optimize_chiller_operation(total_cooling_load_watts, chiller_type="low_temp")

        chiller_preds_low_temp = {}
        if optim_low_ch_count > 0 : # 가동될 때만 예측
            chiller_preds_low_temp = self.chiller_model.predict_pressure_valve(
                total_cooling_load_watts, optim_low_ch_count, chiller_type="low_temp"
            )

        # 4. 고온 냉동기 최적화 및 예측 (예냉 코일 담당)
        optim_high_ch_count, active_high_ids, load_dist_high, power_high, avg_load_high, meets_target_high = \
            self.chiller_optimizer.optimize_chiller_operation(total_precooling_load_watts, chiller_type="high_temp")

        chiller_preds_high_temp = {}
        if optim_high_ch_count > 0 : # 가동될 때만 예측
            chiller_preds_high_temp = self.chiller_model.predict_pressure_valve(
                total_precooling_load_watts, optim_high_ch_count, chiller_type="high_temp"
            )

        # 5. 냉각탑 부하 계산 (총 방열량 = 총 냉각부하 + 총 냉동기 소비전력)
        total_heat_rejection_watts = total_precooling_load_watts + total_cooling_load_watts + power_low + power_high

        # 6. 냉각탑 최적 가동 대수
        active_tower_ids, tower_fan_power, tower_total_capacity, active_tower_count = \
            self.cooling_tower_optimizer.optimize_tower_operation(total_heat_rejection_watts)

        return {
            "oac_predictions_per_unit": oac_results_single_unit,
            "calculated_loads": {
                "total_precooling_load_watts": total_precooling_load_watts, # 고온 냉동기 부하
                "total_cooling_load_watts": total_cooling_load_watts,       # 저온 냉동기 부하
            },
            "low_temp_chiller_optimization": { # 저온 냉동기 (Cooling Coil)
                "optimal_active_count": optim_low_ch_count,
                "active_chiller_ids": active_low_ids,
                "load_distribution_percentage_per_active_chiller": load_dist_low,
                "total_estimated_power_watts": power_low,
                "average_load_percentage_of_active_chillers": avg_load_low,
                "meets_target_load_rate": meets_target_low,
                "predictions": chiller_preds_low_temp
            },
            "high_temp_chiller_optimization": { # 고온 냉동기 (Precooling Coil)
                "optimal_active_count": optim_high_ch_count,
                "active_chiller_ids": active_high_ids,
                "load_distribution_percentage_per_active_chiller": load_dist_high,
                "total_estimated_power_watts": power_high,
                "average_load_percentage_of_active_chillers": avg_load_high,
                "meets_target_load_rate": meets_target_high,
                "predictions": chiller_preds_high_temp
            },
            "cooling_tower_optimization": {
                "total_heat_rejection_load_watts": total_heat_rejection_watts,
                "active_tower_ids": active_tower_ids,
                "active_tower_count": active_tower_count,
                "total_fan_power_watts": tower_fan_power,
                "total_active_capacity_watts": tower_total_capacity
            }
        }

if __name__ == '__main__':
    # 1. 더미 데이터 생성
    print("--- 1. 더미 데이터 생성 중 ---")
    dummy_df = generate_dummy_data(num_rows=2000) # 학습 데이터 양 증가 (2000개 시간별 데이터)
    print(f"더미 데이터 생성 완료. Shape: {dummy_df.shape}")
    # print("더미 데이터 샘플:")
    # print(dummy_df.head())
    # print("\n더미 데이터 컬럼 정보:")
    # print(dummy_df.info())


    # 2. 모델 객체 생성 및 학습
    print("\n--- 2. 모델 학습 진행 ---")
    oac_model_inst = OACModel()
    oac_model_inst.train_coil_models(dummy_df.copy()) # 원본 df 변경 방지

    chiller_model_inst = ChillerModel()
    chiller_model_inst.train_pressure_valve_models(dummy_df.copy())

    # 3. 학습된 모델을 사용하여 Facade 객체 생성
    print("\n--- 3. HVAC 시스템 Facade 생성 (학습된 모델 사용) ---")
    hvac_system = HVACSystemFacade(oac_model_instance=oac_model_inst,
                                   chiller_model_instance=chiller_model_inst)
    print("Facade 객체 생성 완료.")

    # 4. 학습된 모델로 예측 및 최적화 실행 (하절기 시나리오 예시)
    print("\n" + "="*20 + " 학습된 모델 기반 하절기 시나리오 실행 " + "="*20)
    outdoor_temp_summer = 32.0  # 섭씨 (여름철 높은 온도)
    outdoor_humidity_summer = 75.0  # % (여름철 높은 습도)

    oac_target_set_points_summer = {
        'preheating': 18.0,      # 외기가 충분히 높으면 거의 작동 안함
        'precooling': 20.0,      # 외기 예비 냉각 목표 온도
        'cooling': 12.0,         # 주 냉각 코일 목표 온도 (실내 공급 온도보다 낮게 설정)
        'reheating': 23.0,       # 제습 후 실내 공급 온도 조절 목표
        'humidification_rate': 0.0 # 하절기 가습 안함 (개도율 0)
    }
    chilled_water_supply_temp_set_summer = 7.0 # 냉수 공급 설정 온도 (참고용)

    print(f"\n[입력 조건]")
    print(f"  외기 온도: {outdoor_temp_summer}°C, 외기 상대 습도: {outdoor_humidity_summer}%")
    print(f"  OAC 코일별 목표 설정:")
    for coil, sp_val in oac_target_set_points_summer.items():
        if coil != 'humidification_rate':
            print(f"    - {coil.capitalize()} 목표 온도: {sp_val}°C")
        else:
            print(f"    - Humidification 목표 개도율: {sp_val*100:.1f}%")
    print(f"  설정된 냉수 공급 온도 (참고): {chilled_water_supply_temp_set_summer}°C")
    print("-" * 50)

    summer_results = hvac_system.predict_and_optimize(
        outdoor_temp_summer,
        outdoor_humidity_summer,
        oac_target_set_points_summer,
        chilled_water_supply_temp_set_summer
    )

    print("\n[OAC 코일별 예측 결과 (단일 유닛 기준) - 학습된 모델 사용]")
    oac_preds = summer_results["oac_predictions_per_unit"]
    for coil_name in oac_model_inst.coil_order: # 정의된 순서대로 출력
        data = oac_preds.get(coil_name)
        if isinstance(data, dict):
            print(f"  코일: {coil_name.capitalize()}")
            target_display = data.get('target_outlet_temp_c', 'N/A')
            if coil_name == 'humidification': # 가습 코일은 목표가 개도율
                 target_display = f"Rate {oac_target_set_points_summer.get('humidification_rate',0.0)*100:.1f}%"

            print(f"    - 목표 설정: {target_display}")
            print(f"    - 예측 개도율: {data.get('open_rate', 0.0):.3f} ({(data.get('open_rate', 0.0)*100):.1f}%)")
            print(f"    - 입구 조건: {data.get('inlet_temp_c'):.1f}°C, {data.get('inlet_rh_percent'):.1f}%, ({data.get('inlet_h_j_kg'):.0f} J/kg)")
            print(f"    - 출구 조건: {data.get('outlet_temp_c'):.1f}°C, {data.get('outlet_rh_percent'):.1f}%, ({data.get('outlet_h_j_kg'):.0f} J/kg)")
            print(f"    - 엔탈피 변화량: {data.get('delta_h_j_kg'):.0f} J/kg")
    print(f"  최종 토출 조건 (공조기 출구): {oac_preds.get('final_outlet_temp_c'):.1f}°C, {oac_preds.get('final_outlet_rh_percent'):.1f}% ({oac_preds.get('final_outlet_h_j_kg'):.0f} J/kg)")

    print("\n[계산된 총 부하 (OAC 14대 기준)]")
    loads = summer_results["calculated_loads"]
    print(f"  총 예냉 코일 부하 (고온 냉동기 담당): {loads['total_precooling_load_watts']:,.0f} Watts")
    print(f"  총 주 냉각 코일 부하 (저온 냉동기 담당): {loads['total_cooling_load_watts']:,.0f} Watts")

    print("\n[저온 냉동기 최적화 및 예측 - 학습된 모델 사용]")
    low_temp_ch_opt = summer_results["low_temp_chiller_optimization"]
    print(f"  최적 가동 대수: {low_temp_ch_opt['optimal_active_count']} 대 (IDs: {low_temp_ch_opt['active_chiller_ids']})")
    print(f"  가동 냉동기 평균 부하율: {low_temp_ch_opt['average_load_percentage_of_active_chillers']:.2%}")
    print(f"     (목표 55% 근접 여부: {'달성' if low_temp_ch_opt['meets_target_load_rate'] else '미달성/초과'})")
    if low_temp_ch_opt['load_distribution_percentage_per_active_chiller']:
        for i, load_perc in enumerate(low_temp_ch_opt['load_distribution_percentage_per_active_chiller']):
            ch_id = low_temp_ch_opt['active_chiller_ids'][i]
            print(f"    - 냉동기 ID {ch_id} 부하율: {load_perc:.2%}")
    print(f"  예상 총 소비전력: {low_temp_ch_opt['total_estimated_power_watts']:,.0f} Watts")
    print(f"  예측된 차압/밸브 상태: {low_temp_ch_opt['predictions']}")

    print("\n[고온 냉동기 최적화 및 예측 - 학습된 모델 사용]")
    high_temp_ch_opt = summer_results["high_temp_chiller_optimization"]
    print(f"  최적 가동 대수: {high_temp_ch_opt['optimal_active_count']} 대 (IDs: {high_temp_ch_opt['active_chiller_ids']})")
    print(f"  가동 냉동기 평균 부하율: {high_temp_ch_opt['average_load_percentage_of_active_chillers']:.2%}")
    print(f"     (목표 55% 근접 여부: {'달성' if high_temp_ch_opt['meets_target_load_rate'] else '미달성/초과'})")
    if high_temp_ch_opt['load_distribution_percentage_per_active_chiller']:
        for i, load_perc in enumerate(high_temp_ch_opt['load_distribution_percentage_per_active_chiller']):
            ch_id = high_temp_ch_opt['active_chiller_ids'][i]
            print(f"    - 냉동기 ID {ch_id} 부하율: {load_perc:.2%}")
    print(f"  예상 총 소비전력: {high_temp_ch_opt['total_estimated_power_watts']:,.0f} Watts")
    print(f"  예측된 차압/밸브 상태: {high_temp_ch_opt['predictions']}")

    print("\n[냉각탑 최적화 결과]")
    ct_opt = summer_results["cooling_tower_optimization"]
    print(f"  총 방열 부하 (냉동기 부하 + 냉동기 소비전력): {ct_opt['total_heat_rejection_load_watts']:,.0f} Watts")
    print(f"  최적 가동 대수: {ct_opt['active_tower_count']} 대 (IDs: {ct_opt['active_tower_ids']})")
    print(f"  총 팬 소비 전력: {ct_opt['total_fan_power_watts']:,.0f} Watts")
    print(f"  가동 냉각탑 총 방열 용량: {ct_opt['total_active_capacity_watts']:,.0f} Watts")
    if ct_opt['total_active_capacity_watts'] < ct_opt['total_heat_rejection_load_watts'] and ct_opt['total_heat_rejection_load_watts'] > 0 :
        print(f"  경고: 냉각탑 용량이 부족할 수 있습니다. (필요: {ct_opt['total_heat_rejection_load_watts']:,.0f} W, 확보: {ct_opt['total_active_capacity_watts']:,.0f} W)")
    print("="*50)

--- 1. 더미 데이터 생성 중 ---
더미 데이터 생성 완료. Shape: (2000, 31)

--- 2. 모델 학습 진행 ---

--- OAC 코일 모델 학습 시작 ---
  preheating 코일 모델 학습 중...


<ipython-input-1-1e35d38acef8>:184: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = temp_df_train[current_features].fillna(method='ffill').fillna(method='bfill') # 결측치 처리


    preheating 모델 학습 완료. Test MSE: 0.0059
  precooling 코일 모델 학습 중...


<ipython-input-1-1e35d38acef8>:184: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = temp_df_train[current_features].fillna(method='ffill').fillna(method='bfill') # 결측치 처리


    precooling 모델 학습 완료. Test MSE: 0.0019
  cooling 코일 모델 학습 중...


...

    humidification 모델 학습 완료. Test MSE: 0.0005
--- OAC 코일 모델 학습 완료 ---

--- 냉동기 차압/밸브 모델 학습 시작 ---
  low_temp 냉동기 모델 학습 중...


<ipython-input-1-1e35d38acef8>:373: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = df_train_data[features_cols].fillna(method='ffill').fillna(method='bfill')


    low_temp_main_pressure_bar 모델 학습 완료. Test MSE: 0.0099
    low_temp_sub_pressure_valve_rate_percent 모델 학습 완료. Test MSE: 26.1528
    low_temp_main_control_valve_rate_percent 모델 학습 완료. Test MSE: 25.3738
  high_temp 냉동기 모델 학습 중...


<ipython-input-1-1e35d38acef8>:373: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = df_train_data[features_cols].fillna(method='ffill').fillna(method='bfill')


    high_temp_main_pressure_bar 모델 학습 완료. Test MSE: 0.0098
    high_temp_sub_pressure_valve_rate_percent 모델 학습 완료. Test MSE: 26.5190
    high_temp_main_control_valve_rate_percent 모델 학습 완료. Test MSE: 24.9287
--- 냉동기 차압/밸브 모델 학습 완료 ---

--- 3. HVAC 시스템 Facade 생성 (학습된 모델 사용) ---
Facade 객체 생성 완료.

==================== 학습된 모델 기반 하절기 시나리오 실행 ====================

[입력 조건]
  외기 온도: 32.0°C, 외기 상대 습도: 75.0%
  OAC 코일별 목표 설정:
    - Preheating 목표 온도: 18.0°C
    - Precooling 목표 온도: 20.0°C
    - Cooling 목표 온도: 12.0°C
    - Reheating 목표 온도: 23.0°C
    - Humidification 목표 개도율: 0.0%
  설정된 냉수 공급 온도 (참고): 7.0°C
--------------------------------------------------

[OAC 코일별 예측 결과 (단일 유닛 기준) - 학습된 모델 사용]
  코일: Preheating
    - 목표 설정: 18.0
    - 예측 개도율: 0.000 (0.0%)
    - 입구 조건: 32.0°C, 75.0%, (66990 J/kg)
    - 출구 조건: 33.6°C, 58.2%, (66990 J/kg)
    - 엔탈피 변화량: 0 J/kg
  코일: Precooling
    - 목표 설정: 20.0
    - 예측 개도율: 0.884 (88.4%)
    - 입구 조건: 33.6°C, 58.2%, (66990 J/kg)
    - 출구 조건: 31.2°C, 37.7%, (49302 J/kg)